In [1]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google'

In [ ]:
cd drive/My\ Drive/Colab\ Notebooks/Caption-Translation/

/content/drive/My Drive/Colab Notebooks/Caption-Translation


# Start here if not using Colab

In [1]:
import nltk
nltk.download('punkt')


CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]


[nltk_data] Downloading package punkt to /Users/gautam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
from gensim.models import Word2Vec
from matplotlib import pyplot
import nltk
from nltk.tokenize import word_tokenize
import ssl
import gensim
import gensim.downloader
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
import numpy as np


class CaptionScore:
    model = None
    verbose = False

    def __init__(self, num_words = 50000):
        self.model = KeyedVectors.load_word2vec_format("word2vec_pretrained.txt", limit=num_words, binary=False)


    def get_score(self, candidate, objects): # candidate is string caption; objects is the list of objects
        score = 0
        for objectId in objects:
            if CLASSES[objectId]=="N/A":
                continue
            tokens = word_tokenize(candidate)
            tokens = [token.lower() for token in tokens]
            if(CLASSES[objectId] in tokens):
                score += 1
            else:
                max_sim = 0
                max_token = None
                for token in tokens:
                    if(len(token) > 3 or token.isalpha()):
                        new_sim = 0
                        try:
                            new_sim = model.model.similarity(token, CLASSES[objectId])
                        except:
                            pass
                        if (new_sim > max_sim):
                            max_sim = new_sim
                            max_token = token
                if self.verbose:
                      print("Fitted object '", CLASSES[objectId], "' to '", max_token, "' with similarity", max_sim)
                score += max_sim
        return score
    
    def get_score_relu(self, candidate, objects): # candidate is string caption; objects is the list of objects
        score = 0
        for objectId in objects:
            if CLASSES[objectId]=="N/A":
                continue
            tokens = word_tokenize(candidate)
            tokens = [token.lower() for token in tokens]
            if(CLASSES[objectId] in tokens):
                score += 1
            else:
                sum_sim = 0
                for token in tokens:
                    if(len(token) > 3 or token.isalpha()):
                        new_sim = 0
                        try:
                            new_sim = model.model.similarity(token, CLASSES[objectId])
                            sum_sim += max(0.6, new_sim)
                        except:
                            pass
                if self.verbose:
                    print("Fitted object '", CLASSES[objectId], "' to '", max_token, "' with similarity", max_sim)
                score += sum_sim / len(tokens)
        return score
    
    def get_score_arctanh(self, candidate, objects):
        score = 0
        for objectId in objects:
            if CLASSES[objectId]=="N/A":
                continue
            tokens = word_tokenize(candidate)
            tokens = [token.lower() for token in tokens]
            if(CLASSES[objectId] in tokens):
                score += 1
            else:
                sum_sim = 0
                for token in tokens:
                    if(len(token) > 3 or token.isalpha()):
                        new_sim = 0
                        try:
                            new_sim = model.model.similarity(token, CLASSES[objectId])
                            sum_sim += np.arctanh(new_sim)
                        except:
                            pass
                if self.verbose:
                    print("Fitted object '", CLASSES[objectId], "' to '", max_token, "' with similarity", max_sim)
                score += sum_sim / len(tokens)
        return score
    
    def get_score_average(self, candidate, objects):
        score = 0
        for objectId in objects:
            if CLASSES[objectId]=="N/A":
                continue
            tokens = word_tokenize(candidate)
            tokens = [token.lower() for token in tokens]
            if(CLASSES[objectId] in tokens):
                score += 1
            else:
                sum_sim = 0
                for token in tokens:
                    if(len(token) > 3 or token.isalpha()):
                        new_sim = 0
                        try:
                            new_sim = model.model.similarity(token, CLASSES[objectId])
                            sum_sim += new_sim
                        except:
                            pass
                if self.verbose:
                    print("Fitted object '", CLASSES[objectId], "' to '", max_token, "' with similarity", max_sim)
                score += sum_sim / len(tokens)
        return score
    
    def get_score_rms(self, candidate, objects):
        score = 0
        for objectId in objects:
            if CLASSES[objectId]=="N/A":
                continue
            tokens = word_tokenize(candidate)
            tokens = [token.lower() for token in tokens]
            if(CLASSES[objectId] in tokens):
                score += 1
            else:
                sum_sim = 0
                for token in tokens:
                    if(len(token) > 3 or token.isalpha()):
                        new_sim = 0
                        try:
                            new_sim = model.model.similarity(token, CLASSES[objectId])
                            sum_sim += new_sim ** 2
                        except:
                            pass
                if self.verbose:
                    print("Fitted object '", CLASSES[objectId], "' to '", max_token, "' with similarity", max_sim)
                score += (sum_sim / len(tokens)) ** 0.5
        return score

In [72]:
model = CaptionScore()

In [46]:
print(model.model.similarity('person', 'the'))

0.3807391


In [73]:
import json
import random
import sacrebleu

x = json.load(open('data.json', 'r'))

selectedCaptions = []
refs = []

for (index, row) in enumerate(x):
    if(index % 1000 == 0):
        print(index)
    
    if row['objects'][0] == 0:
        pass
    else:
        ibm_caption = row['translations']['ibm']
        azure_caption = row['translations']['azure']

        ibm_score = model.get_score_relu(ibm_caption, set(row['objects']))
        azr_score = model.get_score_relu(azure_caption, set(row['objects']))

        r = {'ibm': row['translations']['ibm'], 'azure': row['translations']['azure']}

    #     if(ibm_score != azr_score):
            #print(row)

        if ibm_score > azr_score:
            r['selected'] = 'ibm'
        elif ibm_score == azr_score:
            r['selected'] = 'ibm'
        else:
            r['selected'] = 'azure'

        ref = row['caption_en']
        refs.append(ref)

        azureScoreBleu = sacrebleu.corpus_bleu([azure_caption], [[ref]])
        ibmScoreBleu = sacrebleu.corpus_bleu([ibm_caption], [[ref]])

        if ibmScoreBleu.score > azureScoreBleu.score:
            r['bleuSelected'] = 'ibm'
        elif ibmScoreBleu.score == azureScoreBleu.score:
            r['bleuSelected'] = 'ibm'
        else:
            r['bleuSelected'] = 'azure'

        selectedCaptions.append(r)
    
    
    # else:
    #   selectedCaptions.append(random.choice(['ibm', 'azr']))
    #print('')
    #print(row['objects'])
    #print('Image:', row['filename'], 'IBM:', model.get_score(row['translations']['ibm'], set(row['objects'])), 'AZURE:', model.get_score(row['translations']['azure'], set(row['objects'])))




0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [74]:
azureBleu = sacrebleu.corpus_bleu([a['azure'] for a in selectedCaptions], [refs])
print("Azure: " + str(azureBleu.score))

ibmBleu = sacrebleu.corpus_bleu([a['ibm'] for a in selectedCaptions], [refs])
print("IBM: " + str(ibmBleu.score))

selectionBleu = sacrebleu.corpus_bleu([a[a['selected']] for a in selectedCaptions], [refs])
print("Combined: " + str(selectionBleu.score))

correctSelectionBleu = sacrebleu.corpus_bleu([a[a['bleuSelected']] for a in selectedCaptions], [refs])
print("Correct Selections: " + str(correctSelectionBleu.score))

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


Azure: 32.41654720349349
IBM: 33.535709688035226
Combined: 32.40535495432709
Correct Selections: 37.60152861508464
